In [ ]:
# Install necessary libraries
!pip install sentence-transformers
!pip install rdflib
!pip install langchain_community
!pip install fitz
!pip install openai==0.28
!pip install owlrl
!pip install tools
!pip install PyMuPDF
!pip install fuzzywuzzy
!pip install python-Levenshtein

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 2.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 97.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 77.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 55.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 12.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 77.5 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstalling nvidia-nvjitlink-cu12-12.5.82:
      Successfully uninstalled nvidia-nvjitlin

In [ ]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [ ]:
# Import required libraries
import os
import tools
import fitz  # PyMuPDF
import re
import openai
import torch
import rdflib
from rdflib import Graph, URIRef, Namespace, RDF, OWL, RDFS, Literal
from rdflib.namespace import XSD
from sentence_transformers import SentenceTransformer, util
from transformers import AutoTokenizer, AutoModelForCausalLM
from owlrl import DeductiveClosure, OWLRL_Semantics
from nltk.corpus import stopwords
import fitz  # PyMuPDF
from urllib.parse import quote
from fuzzywuzzy import process  # For fuzzy matching
from google.colab import userdata
import pandas as pd

# Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')

# Mount API KEY
os.environ['OPENAI_API_KEY'] = userdata.get('OPENAI_API_KEY')
openai.api_key = os.environ['OPENAI_API_KEY']

Mounted at /content/drive


### Retrieve Classes, Properties from existing Ontology:

In [ ]:
from rdflib import Graph, Namespace, RDF, OWL

# Load ontology
ontology_data = Graph()
ontology_data.parse('/content/drive/MyDrive/KG_data/hi_ontology_final_1.ttl', format='turtle')


# Define namespace (Assuming HI is the correct ontology namespace)
HI = Namespace("http://www.semanticweb.org/hi_ontology#")

# Function to extract local names from URIs
def extract_local_name(uri):
    """Extracts the local name after '#' or last '/' from a URI"""
    return uri.split("#")[-1] if "#" in uri else uri.split("/")[-1]

# Extract existing ontology details
existing_classes = [
    extract_local_name(cls) for cls in ontology_data.subjects(RDF.type, OWL.Class)
]

existing_object_property = [
    extract_local_name(object_property) for object_property in ontology_data.subjects(RDF.type, OWL.ObjectProperty)
]

existing_individual = [
    extract_local_name(individual) for individual in ontology_data.subjects(RDF.type, OWL.NamedIndividual)
]

# Display ontology structure
print(f'Existing Ontology is comprised of:\n Classes: {existing_classes}\n Properties: {existing_object_property}\n Individuals: {existing_individual}')
print(f'Existing Ontology count is:\n Total Classes: {len(existing_classes)}\n Total Properties: {len(existing_object_property)}\n Total Individuals: {len(existing_individual)}')


Existing Ontology is comprised of:
 Classes: ['AccidentalDeath', 'AttemptedCrime', 'ComplicityInMurder', 'CrimeMethod', 'CrimeMotive', 'CrimeScene', 'Evidence', 'EvidenceArgument', 'ForensicAnalysis', 'Intent', 'LegalArgument', 'LegalConsideration', 'LegalFact', 'LegalJustification', 'LegalPrinciple', 'LegalScheme', 'n198a7db939d54e09aba61c24e0e10970b1', 'Manslaughter', 'Murder', 'NegligentHomicide', 'PhysicalEvidence', 'Testimony', 'VictimSuspectRelationship', 'Weapon', 'WeaponPosession', 'BayesianFragment', 'ComplexProblem', 'GameModelingSystem', 'CulturalConsiderations', 'EthicalDecisionModel', 'EthicalExperiment', 'EthicsResearcher', 'ExplainabilityModel', 'JustificationParameters', 'LegalConsideration', 'PersonalValues', 'ProposedAction', 'ChatGPTEducationalConsequence', 'CommunicationChannel', 'SearchEngine', 'Teacher', 'SkinLesion', 'AIMethodology', 'AIPractitioner', 'AIRefinementProcess', 'Accomplice', 'Alibi', 'AnswerMethod', 'ArtificialAgentEndGoal', 'BayesianLayer', 'Cancer'

### Try NER using whole Research Paper:


*   Retrieve Terms(ner-value) from Research paper such that it relates to the group of exisitng classes (ner_entity_group)
*   Create a dataframe having Paper_id, Paper_Title, Author, ner-value, ner-entity-group



In [ ]:
# Define function for NER Retrieve Terms(ner-value) from Research paper such that it relates to the group of exisitng classes (ner_entity_group)
# Global DataFrame for extracted entities
global extracted_entities_df
extracted_entities_df = pd.DataFrame(columns=["paper_id", "paper_title", "author", "ner_value", "ner_entity_group"])

def extract_terms_from_text(full_text, existing_classes):
    """
    Uses GPT-4 to extract relevant NER terms, map them to ontology-based categories,
    remove numbers, filter out author names, and include confidence scores.
    """
    response = openai.ChatCompletion.create(
        model="gpt-4-turbo",
        messages=[
            {
                "role": "system",
                "content": (
                    "You are an expert in ontology enrichment and named entity recognition (NER). "
                    "Your task is to extract key terms from research papers and classify them into predefined groups. "
                    "Ensure that you follow these strict instructions:\n"
                    "1. **Extract only relevant key terms related to the given ontology categories**.\n"
                    "2. **Do NOT include author names, affiliations, or references in the extracted terms**.\n"
                    "3. **Give only terms and not phrases with more than 3 terms"
                    "3. **Exclude any term that contains numbers (e.g., 'GPT-4', '2023')**.\n"
                    "4. **For each extracted term, provide a confidence score between 0 and 1, where 1 is the highest confidence**.\n"
                    "5. **Strictly format the output as follows**:\n\n"
                    "Format:\n"
                    "<NER_Value> - <Entity_Group> - <Confidence_Score>\n"
                    "<NER_Value> - <Entity_Group> - <Confidence_Score>\n"
                    "..."
                )
            },
            {
                "role": "user",
                "content": (
                    f"Extract structured terms from this research paper text and classify them into relevant ontology categories.\n\n"
                    f"Text:\n{full_text}\n\n"
                    f"Ontology Categories:\n{', '.join(existing_classes)}\n\n"
                    "Ensure your response follows the specified format strictly."
                )
            }
        ]
    )

    extracted_data = response['choices'][0]['message']['content'].strip()

    # Convert extracted text into a structured format
    entity_data = []
    for line in extracted_data.split("\n"):
        if " - " in line:
            parts = [part.strip() for part in line.split(" - ")]
            if len(parts) == 3:
                ner_value, ner_entity_group, ner_score = parts

                # Ensure ner_score is numeric
                try:
                    ner_score = float(ner_score)
                except ValueError:
                    ner_score = None  # Assign None if parsing fails

                entity_data.append({"ner_value": ner_value, "ner_entity_group": ner_entity_group, "ner_score": ner_score})

    return entity_data  # Return list of dictionaries


In [ ]:
# Parse all the research paper and save them as a data frame with columns as paper_id - file name,paper_title, author, extracted_text
def process_single_paper(pdf_path):
    """
    Extracts text from a research paper and applies LLM-based NER.
    """
    global extracted_entities_df  # Ensure global access

    # Extract paper_id and metadata
    paper_id = os.path.basename(pdf_path)

    # Open and extract text from the PDF
    doc = fitz.open(pdf_path)
    extracted_text = "".join([page.get_text() for page in doc])
    doc.close()

    # Extract title and author heuristically
    first_page_text = extracted_text.split("\n")[:10]  # Get first few lines for metadata
    paper_title = first_page_text[1] if len(first_page_text) > 0 else "Unknown Title"
    author = first_page_text[0] if len(first_page_text) > 1 else "Unknown Author"

    # Apply GPT-based NER using extracted ontology classes
    extracted_entities = extract_terms_from_text(extracted_text, existing_classes)

    # Convert extracted entities into DataFrame
    entity_data = [
        {"paper_id": paper_id, "paper_title": paper_title, "author": author, **entity}
        for entity in extracted_entities
    ]

    # Append new data to the global DataFrame
    extracted_entities_df = pd.concat([extracted_entities_df, pd.DataFrame(entity_data)], ignore_index=True)

    return extracted_entities_df  # Return updated DataFrame

In [ ]:
process_single_paper("/content/drive/MyDrive/KG_data/multiple_papers/2305.05422v1.pdf")

,paper_id,paper_title,author,ner_value,ner_entity_group,ner_score
0,2305.05422v1.pdf,Egocentric Hierarchical Visual Semantics,L. Erculiani et al. /,object recognition,AIMethodology,0.95
1,2305.05422v1.pdf,Egocentric Hierarchical Visual Semantics,L. Erculiani et al. /,lexical semantics,Discipline,0.92
2,2305.05422v1.pdf,Egocentric Hierarchical Visual Semantics,L. Erculiani et al. /,visual semantics,Topic,0.91
3,2305.05422v1.pdf,Egocentric Hierarchical Visual Semantics,L. Erculiani et al. /,interactive machine learning,Interaction,0.90
4,2305.05422v1.pdf,Egocentric Hierarchical Visual Semantics,L. Erculiani et al. /,hierarchical semantic structure,ProblemCharacterstic,0.93
5,2305.05422v1.pdf,Egocentric Hierarchical Visual Semantics,L. Erculiani et al. /,algorithm,System,0.90
6,2305.05422v1.pdf,Egocentric Hierarchical Visual Semantics,L. Erculiani et al. /,Teleosemantics,Discipline,0.80
7,2305.05422v1.pdf,Egocentric Hierarchical Visual Semantics,L. Erculiani et al. /,visual genus,ProblemView,0.88
8,2305.05422v1.pdf,Egocentric Hierarchical Visual Semantics,L. Erculiani et al. /,visual differentia,ProblemView,0.88
9,2305.05422v1.pdf,Egocentric Hierarchical Visual Semantics,L. Erculiani et al. /,egocentric point-of-view,Perspective,0.85


In [ ]:
process_single_paper("/content/drive/MyDrive/KG_data/multiple_papers/A_Conceptual_Model_for_Implementing_Explainable_AI.pdf")

,paper_id,paper_title,author,ner_value,ner_entity_group,ner_score
0,2305.05422v1.pdf,Egocentric Hierarchical Visual Semantics,L. Erculiani et al. /,object recognition,AIMethodology,0.95
1,2305.05422v1.pdf,Egocentric Hierarchical Visual Semantics,L. Erculiani et al. /,lexical semantics,Discipline,0.92
2,2305.05422v1.pdf,Egocentric Hierarchical Visual Semantics,L. Erculiani et al. /,visual semantics,Topic,0.91
3,2305.05422v1.pdf,Egocentric Hierarchical Visual Semantics,L. Erculiani et al. /,interactive machine learning,Interaction,0.90
4,2305.05422v1.pdf,Egocentric Hierarchical Visual Semantics,L. Erculiani et al. /,hierarchical semantic structure,ProblemCharacterstic,0.93
5,2305.05422v1.pdf,Egocentric Hierarchical Visual Semantics,L. Erculiani et al. /,algorithm,System,0.90
6,2305.05422v1.pdf,Egocentric Hierarchical Visual Semantics,L. Erculiani et al. /,Teleosemantics,Discipline,0.80
7,2305.05422v1.pdf,Egocentric Hierarchical Visual Semantics,L. Erculiani et al. /,visual genus,ProblemView,0.88
8,2305.05422v1.pdf,Egocentric Hierarchical Visual Semantics,L. Erculiani et al. /,visual differentia,ProblemView,0.88
9,2305.05422v1.pdf,Egocentric Hierarchical Visual Semantics,L. Erculiani et al. /,egocentric point-of-view,Perspective,0.85


In [ ]:
process_single_paper("/content/drive/MyDrive/KG_data/multiple_papers/Legal-Ethical_Challenges_and_Technological_Solutio.pdf")

,paper_id,paper_title,author,ner_value,ner_entity_group,ner_score
0,2305.05422v1.pdf,Egocentric Hierarchical Visual Semantics,L. Erculiani et al. /,object recognition,AIMethodology,0.95
1,2305.05422v1.pdf,Egocentric Hierarchical Visual Semantics,L. Erculiani et al. /,lexical semantics,Discipline,0.92
2,2305.05422v1.pdf,Egocentric Hierarchical Visual Semantics,L. Erculiani et al. /,visual semantics,Topic,0.91
3,2305.05422v1.pdf,Egocentric Hierarchical Visual Semantics,L. Erculiani et al. /,interactive machine learning,Interaction,0.90
4,2305.05422v1.pdf,Egocentric Hierarchical Visual Semantics,L. Erculiani et al. /,hierarchical semantic structure,ProblemCharacterstic,0.93
...,...,...,...,...,...,...
84,Legal-Ethical_Challenges_and_Technological_Sol...,Technological Solutions to e-Health Data,Legal-Ethical Challenges and,consent policies,LegalScheme,0.88
85,Legal-Ethical_Challenges_and_Technological_Sol...,Technological Solutions to e-Health Data,Legal-Ethical Challenges and,digital interactions,InteractionTask,0.87
86,Legal-Ethical_Challenges_and_Technological_Sol...,Technological Solutions to e-Health Data,Legal-Ethical Challenges and,autonomous digital interactions,InteractionTask,0.88
87,Legal-Ethical_Challenges_and_Technological_Sol...,Technological Solutions to e-Health Data,Legal-Ethical Challenges and,privacy preferences,PersonalValues,0.91


In [ ]:
process_single_paper("/content/drive/MyDrive/KG_data/multiple_papers/Open_Multiple_Adjunct_Decision_Support_at_the_Time.pdf")

,paper_id,paper_title,author,ner_value,ner_entity_group,ner_score
0,2305.05422v1.pdf,Egocentric Hierarchical Visual Semantics,L. Erculiani et al. /,object recognition,AIMethodology,0.95
1,2305.05422v1.pdf,Egocentric Hierarchical Visual Semantics,L. Erculiani et al. /,lexical semantics,Discipline,0.92
2,2305.05422v1.pdf,Egocentric Hierarchical Visual Semantics,L. Erculiani et al. /,visual semantics,Topic,0.91
3,2305.05422v1.pdf,Egocentric Hierarchical Visual Semantics,L. Erculiani et al. /,interactive machine learning,Interaction,0.90
4,2305.05422v1.pdf,Egocentric Hierarchical Visual Semantics,L. Erculiani et al. /,hierarchical semantic structure,ProblemCharacterstic,0.93
...,...,...,...,...,...,...
109,Open_Multiple_Adjunct_Decision_Support_at_the_...,at the Time of Relational AI,"Open, Multiple, Adjunct. Decision Support",Artificial Intelligence,AIMethodology,0.95
110,Open_Multiple_Adjunct_Decision_Support_at_the_...,at the Time of Relational AI,"Open, Multiple, Adjunct. Decision Support",Decision-making process,Process,0.83
111,Open_Multiple_Adjunct_Decision_Support_at_the_...,at the Time of Relational AI,"Open, Multiple, Adjunct. Decision Support",Human cognition,HumanPerformance,0.87
112,Open_Multiple_Adjunct_Decision_Support_at_the_...,at the Time of Relational AI,"Open, Multiple, Adjunct. Decision Support",AI-enabled tools,ArtificialAgent,0.91


In [ ]:
process_single_paper("/content/drive/MyDrive/KG_data/multiple_papers/hhai-2022_paper_67.pdf")

,paper_id,paper_title,author,ner_value,ner_entity_group,ner_score
0,2305.05422v1.pdf,Egocentric Hierarchical Visual Semantics,L. Erculiani et al. /,object recognition,AIMethodology,0.95
1,2305.05422v1.pdf,Egocentric Hierarchical Visual Semantics,L. Erculiani et al. /,lexical semantics,Discipline,0.92
2,2305.05422v1.pdf,Egocentric Hierarchical Visual Semantics,L. Erculiani et al. /,visual semantics,Topic,0.91
3,2305.05422v1.pdf,Egocentric Hierarchical Visual Semantics,L. Erculiani et al. /,interactive machine learning,Interaction,0.90
4,2305.05422v1.pdf,Egocentric Hierarchical Visual Semantics,L. Erculiani et al. /,hierarchical semantic structure,ProblemCharacterstic,0.93
...,...,...,...,...,...,...
124,hhai-2022_paper_67.pdf,Human-Machine intelligence,Knowledge Graphs in support of,Hybrid Intelligence,ResearchStudyEndGoal,0.88
125,hhai-2022_paper_67.pdf,Human-Machine intelligence,Knowledge Graphs in support of,Enterprise Knowledge Graph platform,System,0.80
126,hhai-2022_paper_67.pdf,Human-Machine intelligence,Knowledge Graphs in support of,graph machine learning library,System,0.82
127,hhai-2022_paper_67.pdf,Human-Machine intelligence,Knowledge Graphs in support of,Global Counterfactual Explanation,ExplainabilityModel,0.90


In [ ]:
#convert the dataframe into a csv file and save it
extracted_entities_df.to_csv("/content/drive/MyDrive/KG_data/ner_extracted_entities_df_(2).csv", index=False)

### To check whether the existing individual is same as the terms retrieved

In [ ]:
def check_terms(df, existing_individuals):
    """
    Check and remove rows from the DataFrame where ner_value matches existing individuals.

    Parameters:
        df (pandas.DataFrame): DataFrame containing the retrieved terms.
        existing_individuals (list): List of existing individuals from the ontology.

    Returns:
        pandas.DataFrame: Filtered DataFrame without matched ner_value terms.
    """
    # Convert the list to a set for faster lookup
    existing_individuals_set = set(existing_individuals)

    # Filter the DataFrame to remove rows with ner_value matching existing individuals
    filtered_df = df[~df['ner_value'].isin(existing_individuals_set)].reset_index(drop=True)
    return filtered_df



In [ ]:
filtered_df = check_terms(extracted_entities_df, existing_individual)

In [ ]:
filtered_df.head(100)

,paper_id,paper_title,author,ner_value,ner_entity_group,ner_score
0,2305.05422v1.pdf,Egocentric Hierarchical Visual Semantics,L. Erculiani et al. /,object recognition,AIMethodology,0.95
1,2305.05422v1.pdf,Egocentric Hierarchical Visual Semantics,L. Erculiani et al. /,lexical semantics,Discipline,0.92
2,2305.05422v1.pdf,Egocentric Hierarchical Visual Semantics,L. Erculiani et al. /,visual semantics,Topic,0.91
3,2305.05422v1.pdf,Egocentric Hierarchical Visual Semantics,L. Erculiani et al. /,interactive machine learning,Interaction,0.90
4,2305.05422v1.pdf,Egocentric Hierarchical Visual Semantics,L. Erculiani et al. /,hierarchical semantic structure,ProblemCharacterstic,0.93
...,...,...,...,...,...,...
95,Open_Multiple_Adjunct_Decision_Support_at_the_...,at the Time of Relational AI,"Open, Multiple, Adjunct. Decision Support",Cognitive motivation,HumanBehaviorPrediction,0.75
96,Open_Multiple_Adjunct_Decision_Support_at_the_...,at the Time of Relational AI,"Open, Multiple, Adjunct. Decision Support",Performance improvement,Performance,0.90
97,Open_Multiple_Adjunct_Decision_Support_at_the_...,at the Time of Relational AI,"Open, Multiple, Adjunct. Decision Support",Interaction protocol,InteractionProtocol,0.95
98,Open_Multiple_Adjunct_Decision_Support_at_the_...,at the Time of Relational AI,"Open, Multiple, Adjunct. Decision Support",Decision support systems,System,0.92


In [ ]:
filtered_df.to_csv("/content/drive/MyDrive/KG_data/ner_extracted_entities_df_filtered.csv", index=False)

##Similarity between terms are retrieved with fast text for word embedding and cosine similarity metrics for evaluation in the file finetune_fasttext.ipynb

##Define relationship names for those with higher similarity score (>0.75) with open_api

check whether the existing object property can be applied for developing the relationship between the individuals else you can define new relationships.

check whether the existing object-property can be applied for developing the relationship between the individuals else you can define new relationships. - need to add metrics to check how much it is correct - constraint of having at most 3 relationships for one object property.

Already have 2 dataframes holding:

Term_1	Term_2	Term_1_Paper_ID	Term_2_Paper_ID	Term_1_entity	Term_2_entity	Similarity_Score

other one holding:
NER_Entity	Ontology_Individual	NER_Class	Ontology_Class	Similarity_Score

Finally create a dataframe such that term1,term2,term1_entity,term_2_entity,similarity_score, object_property, term1_paper_id, term_2_paper_id - for new term relations

And for eo and new term create a dataframe such that NER_Entity, Ontology_Individual, NER_Class, object_property, Ontology_Class, Similarity_Score.

In [ ]:
import pandas as pd

# Load the similarity DataFrames
similarity_df = pd.read_csv("/content/drive/MyDrive/KG_data/high_similarity_df.csv")
ner_ontology_similarity_df = pd.read_csv("/content/drive/MyDrive/KG_data/high_similarity_df_eo.csv")

print("Similarity DataFrame:")
print(similarity_df.head())

print("NER and Ontology Similarity DataFrame:")
print(ner_ontology_similarity_df.head())


Similarity DataFrame:
                     Term_1                            Term_2  \
0  egocentric point-of-view  unsupervised deep neural network   
1  egocentric point-of-view               taxonomic structure   
2  egocentric point-of-view           Collaborative diagnosis   
3  egocentric point-of-view             Disease-Symptom links   
4              semantic gap     Hierarchical Visual Semantics   

    Term_1_Paper_ID                                    Term_2_Paper_ID  \
0  2305.05422v1.pdf                                   2305.05422v1.pdf   
1  2305.05422v1.pdf  A_Conceptual_Model_for_Implementing_Explainabl...   
2  2305.05422v1.pdf                             hhai-2022_paper_67.pdf   
3  2305.05422v1.pdf                             hhai-2022_paper_67.pdf   
4  2305.05422v1.pdf                                   2305.05422v1.pdf   

  Term_1_entity      Term_2_entity  Similarity_Score  
0   Perspective      AIMethodology          0.998917  
1   Perspective  CategoryStructu

In [ ]:
print(existing_object_property)

['adaptsPreferencesOn', 'adaptsToUserPreferencesUsing', 'InputsMedicalImagesTo', 'OfferedAt', 'OperatesAs', 'PartOf', 'PowersApplication', 'RequiresCapability', 'UsesModel', 'aimsToMitigate', 'appliesNaturalLanguageUsing', 'assessesCapability', 'attemptsToSolve', 'belongsToDiscipline', 'captures', 'categorizes', 'collaboratesWith', 'comparesWith', 'concludes', 'conductsResearchOn', 'consistOfPerformanceCompoent', 'coversTopic', 'demonstratesCapability', 'describesProblem', 'designs', 'designsTask', 'develops', 'drives', 'employes', 'enhancesReasoningUsing', 'enrolledIn', 'evaluates', 'evaluatesOutput', 'evaluatesSenarioUsing', 'evaluatesUncertainty', 'executesMoralProfilingUsing', 'experiences', 'explainsEthicalDecisionsUsing', 'exposes', 'filtersInformationUsing', 'follows', 'followsStrategy', 'generatesOutput', 'hasComplianceMonitoredBy', 'hasExpert', 'hasExpertiseIn', 'hasFindingsPublishedBy', 'hasImpact', 'hasNegativeImpact', 'hasPerformanceMetric', 'hasPositiveImpact', 'hasProblem

In [ ]:
import openai
import re

def identify_relationship_with_gpt(term1, term2, existing_properties):
    """
    Uses GPT-4 to identify a relationship between two terms.
    Returns the top relationship with the highest confidence score and whether it is existing or new.
    """
    try:
        response = openai.ChatCompletion.create(
            model="gpt-4-turbo",
            messages=[
                {
                    "role": "system",
                    "content": (
                        "You are an expert in ontology-based relationship extraction and validation. "
                        "Your primary task is to identify the relationship between two terms based on the given existing ontology object properties. "
                        "Follow these guidelines strictly:\n"
                        "1. Only use the given existing properties for evaluating the relationship.\n"
                        "2. If a valid relationship is found with a confidence score >0.75, return it directly.\n"
                        "3. If the relationship exists, mark it as 'existing'.\n"
                        "4. If no suitable existing relationship is found, suggest a new relationship with a relevance score and mark it as 'new'.\n"
                        "5. Do not include any explanations or extra text. Return the result in the following strict format:\n\n"
                        "Format:\n"
                        "Object_Property: <relationship_name> - <Confidence_Score> - <Status (existing/new)>\n"
                    )
                },
                {
                    "role": "user",
                    "content": (
                        f"Determine the relationship between the following terms:\n"
                        f"Term 1: {term1}\n"
                        f"Term 2: {term2}\n"
                        f"Existing Object Properties: {', '.join(existing_properties)}\n\n"
                        "If no suitable existing relationship is found, suggest a new relationship with a relevance score."
                    )
                }
            ]
        )

        # Extract the response from GPT-4
        result = response['choices'][0]['message']['content'].strip()

        # Use regex to strictly extract the expected format
        pattern = r"Object_Property:\s*(\w+)\s*-\s*([\d.]+)\s*-\s*(existing|new)"
        match = re.search(pattern, result)

        if match:
            object_property = match.group(1)
            confidence = float(match.group(2))
            status = match.group(3)
            return object_property, confidence, status

        # Fallback if no valid relationship is found
        return "no_relationship", 0.0, "unknown"

    except Exception as e:
        return "no_relationship", 0.0, "unknown"


In [ ]:
# Function to apply relationship extraction to each row
def extract_relationship(row):
    relation, score, status = identify_relationship_with_gpt(row['Term_1'], row['Term_2'], existing_object_property)
    return pd.Series([relation, score, status])

# Apply the function directly to the DataFrame
similarity_df[['object_property', 'rel_confidence_Score', 'relation_status']] = similarity_df.apply(extract_relationship, axis=1)

# Display the updated DataFrame
print("Updated Similarity DataFrame:")
similarity_df.head()

Updated Similarity DataFrame:


,Term_1,Term_2,Term_1_Paper_ID,Term_2_Paper_ID,Term_1_entity,Term_2_entity,Similarity_Score,object_property,rel_confidence_Score,relation_status
1,egocentric point-of-view,taxonomic structure,2305.05422v1.pdf,A_Conceptual_Model_for_Implementing_Explainabl...,Perspective,CategoryStructure,0.999049,isAViewOf,0.85,existing
2,egocentric point-of-view,Collaborative diagnosis,2305.05422v1.pdf,hhai-2022_paper_67.pdf,Perspective,MedicalRole,0.998061,isAViewOf,0.78,existing
5,semantic gap,visual-semantic embeddings,2305.05422v1.pdf,2305.05422v1.pdf,Problem,ImageClassificationModel,0.999452,aimsToMitigate,0.85,existing
6,Hierarchical Visual Semantics,visual-semantic embeddings,2305.05422v1.pdf,2305.05422v1.pdf,Topic,ImageClassificationModel,0.998389,coversTopic,0.80,existing
7,Extreme Value Machines,method engineering,2305.05422v1.pdf,A_Conceptual_Model_for_Implementing_Explainabl...,System,EngineeringMethod,0.998048,coversTopic,0.85,existing


In [ ]:
# remove terms that does not have any any relationship
similarity_df = similarity_df[similarity_df['rel_confidence_Score'] > 0.75]

In [ ]:
similarity_df.head(100)

,Term_1,Term_2,Term_1_Paper_ID,Term_2_Paper_ID,Term_1_entity,Term_2_entity,Similarity_Score,object_property,rel_confidence_Score,relation_status
1,egocentric point-of-view,taxonomic structure,2305.05422v1.pdf,A_Conceptual_Model_for_Implementing_Explainabl...,Perspective,CategoryStructure,0.999049,isAViewOf,0.85,existing
2,egocentric point-of-view,Collaborative diagnosis,2305.05422v1.pdf,hhai-2022_paper_67.pdf,Perspective,MedicalRole,0.998061,isAViewOf,0.78,existing
5,semantic gap,visual-semantic embeddings,2305.05422v1.pdf,2305.05422v1.pdf,Problem,ImageClassificationModel,0.999452,aimsToMitigate,0.85,existing
6,Hierarchical Visual Semantics,visual-semantic embeddings,2305.05422v1.pdf,2305.05422v1.pdf,Topic,ImageClassificationModel,0.998389,coversTopic,0.80,existing
7,Extreme Value Machines,method engineering,2305.05422v1.pdf,A_Conceptual_Model_for_Implementing_Explainabl...,System,EngineeringMethod,0.998048,coversTopic,0.85,existing
9,Extreme Value Machines,Human-centered AI,2305.05422v1.pdf,Open_Multiple_Adjunct_Decision_Support_at_the_...,System,AIRefinementProcess,0.998464,isAssociatedWith,0.80,new
11,Extreme Value Machines,multi-omics data sets,2305.05422v1.pdf,hhai-2022_paper_67.pdf,System,Dataset,0.998674,integratesDataOfType,0.85,existing
14,unsupervised deep neural network,Collaborative diagnosis,2305.05422v1.pdf,hhai-2022_paper_67.pdf,AIMethodology,MedicalRole,0.999094,usesModel,0.80,existing
16,financial services,Dutch financial services,A_Conceptual_Model_for_Implementing_Explainabl...,A_Conceptual_Model_for_Implementing_Explainabl...,Sector,Sector,0.999356,PartOf,0.80,existing
18,taxonomic structure,Disease-Symptom links,A_Conceptual_Model_for_Implementing_Explainabl...,hhai-2022_paper_67.pdf,CategoryStructure,Output,0.998617,conductsResearchOn,0.78,existing


In [ ]:
similarity_df['object_property'].unique()

array(['isAViewOf', 'aimsToMitigate', 'coversTopic', 'isAssociatedWith',
       'integratesDataOfType', 'usesModel', 'PartOf',
       'conductsResearchOn', 'RequiresCapability',
       'adaptsToUserPreferencesUsing', 'uses', 'hasInteraction',
       'collaboratesWith', 'usesDataset', 'isProcessedBy',
       'adaptsPreferencesOn', 'influencesConsequence',
       'supportsPerformanceImprovement', 'attemptsToSolve', 'develops'],
      dtype=object)

In [ ]:
similarity_df['relation_status'].value_counts()

,count
relation_status,
existing,45
new,3


In [ ]:
similarity_df.to_csv("/content/drive/MyDrive/KG_data/Relation_dataset_saved_new_new.csv", index=False)

For existing and new term data:

In [ ]:
ner_ontology_similarity_df.head()

,NER_Entity,Ontology_Individual,NER_Class,Ontology_Class,Similarity_Score
0,egocentric point-of-view,isStrengthensBy,Perspective,ObjectProperty,0.999666
1,egocentric point-of-view,Testimony,Perspective,Class,0.999533
2,unsupervised deep neural network,LMUMunich,AIMethodology,NamedIndividual,0.999661
3,unsupervised deep neural network,AIAssistedDiagnosis,AIMethodology,Output,0.999583
4,unsupervised deep neural network,ExtractedFromChatGPT,AIMethodology,NamedIndividual,0.999725


In [ ]:
# Function to apply relationship extraction to each row
def extract_relationship(row):
    relation, score, status = identify_relationship_with_gpt(row['NER_Entity'], row['Ontology_Individual'], existing_object_property)
    return pd.Series([relation, score, status])

# Apply the function directly to the DataFrame
ner_ontology_similarity_df[['object_property', 'rel_confidence_Score', 'relation_status']] = ner_ontology_similarity_df.apply(extract_relationship, axis=1)

# Display the updated DataFrame
print("Updated NER_ontology Similarity DataFrame:")
ner_ontology_similarity_df.head()

Updated NER_ontology Similarity DataFrame:


,NER_Entity,Ontology_Individual,NER_Class,Ontology_Class,Similarity_Score,object_property,rel_confidence_Score,relation_status
0,egocentric point-of-view,isStrengthensBy,Perspective,ObjectProperty,0.999666,isStrengthensBy,0.78,existing
1,egocentric point-of-view,Testimony,Perspective,Class,0.999533,providesPerspectiveFor,0.82,existing
2,unsupervised deep neural network,LMUMunich,AIMethodology,NamedIndividual,0.999661,isConductedBy,0.78,existing
3,unsupervised deep neural network,AIAssistedDiagnosis,AIMethodology,Output,0.999583,UsesModel,0.85,existing
4,unsupervised deep neural network,ExtractedFromChatGPT,AIMethodology,NamedIndividual,0.999725,usesModel,0.85,existing


In [ ]:
ner_ontology_similarity_df= ner_ontology_similarity_df[ner_ontology_similarity_df['rel_confidence_Score'] > 0.75]

In [ ]:
ner_ontology_similarity_df.head()

,NER_Entity,Ontology_Individual,NER_Class,Ontology_Class,Similarity_Score,object_property,rel_confidence_Score,relation_status
0,egocentric point-of-view,isStrengthensBy,Perspective,ObjectProperty,0.999666,isStrengthensBy,0.78,existing
1,egocentric point-of-view,Testimony,Perspective,Class,0.999533,providesPerspectiveFor,0.82,existing
2,unsupervised deep neural network,LMUMunich,AIMethodology,NamedIndividual,0.999661,isConductedBy,0.78,existing
3,unsupervised deep neural network,AIAssistedDiagnosis,AIMethodology,Output,0.999583,UsesModel,0.85,existing
4,unsupervised deep neural network,ExtractedFromChatGPT,AIMethodology,NamedIndividual,0.999725,usesModel,0.85,existing


In [ ]:
ner_ontology_similarity_df['object_property'].unique()

array(['isStrengthensBy', 'providesPerspectiveFor', 'isConductedBy',
       'UsesModel', 'usesModel', 'designsTask', 'belongsToDiscipline',
       'hasPerformanceMetric', 'measuresPerformanceOfSystem',
       'coversTopic', 'uses', 'adaptsToUserPreferencesUsing',
       'hasCapability', 'supportsPerformanceImprovement', 'performsTask',
       'isCharacterisedAs', 'aimsToMitigate', 'hasNoDirectRelation',
       'inScenario', 'collaboratesWith', 'attemptsToSolve',
       'enhancesReasoningUsing', 'integratesDataOfType',
       'conductsResearchOn', 'tacklesIssuesOf'], dtype=object)

In [ ]:
ner_ontology_similarity_df['relation_status'].value_counts()

,count
relation_status,
existing,39
new,3


In [ ]:
ner_ontology_similarity_df.to_csv("/content/drive/MyDrive/KG_data/Relation_dataset_saved_new_new_eo.csv", index=False)

In [ ]:
ner_ontology_similarity_df.to_excel("/content/drive/MyDrive/KG_data/Relation_dataset_saved_new_new_eo.xlsx", index=False)

In [ ]:
similarity_df.to_excel("/content/drive/MyDrive/KG_data/Relation_dataset_saved_new_new.xlsx", index=False)

## Development of overall Ontology with rdflib

1) Need to add new terms retrieved to be under the individuals
2) create relationships (object_property) retrieved between existing and new terms
3) create relationships between new new terms

In [ ]:
similarity_df = pd.read_csv("/content/drive/MyDrive/KG_data/Final_data/Relation_dataset_saved_new_new.csv")

In [ ]:
similarity_df.head()

,Term_1,Term_2,Term_1_Paper_ID,Term_2_Paper_ID,Term_1_entity,Term_2_entity,Similarity_Score,object_property,rel_confidence_Score,relation_status
0,egocentric point-of-view,taxonomic structure,2305.05422v1.pdf,A_Conceptual_Model_for_Implementing_Explainabl...,Perspective,CategoryStructure,0.999049,isAViewOf,0.85,existing
1,egocentric point-of-view,Collaborative diagnosis,2305.05422v1.pdf,hhai-2022_paper_67.pdf,Perspective,MedicalRole,0.998061,isAViewOf,0.78,existing
2,semantic gap,visual-semantic embeddings,2305.05422v1.pdf,2305.05422v1.pdf,Problem,ImageClassificationModel,0.999452,aimsToMitigate,0.85,existing
3,Hierarchical Visual Semantics,visual-semantic embeddings,2305.05422v1.pdf,2305.05422v1.pdf,Topic,ImageClassificationModel,0.998389,coversTopic,0.80,existing
4,Extreme Value Machines,method engineering,2305.05422v1.pdf,A_Conceptual_Model_for_Implementing_Explainabl...,System,EngineeringMethod,0.998048,coversTopic,0.85,existing


In [ ]:
similarity_df['Term_1_Paper_ID'].unique()

array(['2305.05422v1.pdf',
       'A_Conceptual_Model_for_Implementing_Explainable_AI.pdf',
       'Legal-Ethical_Challenges_and_Technological_Solutio.pdf',
       'Open_Multiple_Adjunct_Decision_Support_at_the_Time.pdf',
       'hhai-2022_paper_67.pdf'], dtype=object)

In [ ]:
ner_ontology_similarity_df = pd.read_csv("/content/drive/MyDrive/KG_data/Final_data/Relation_dataset_saved_new_new_eo.csv")

In [ ]:
ner_ontology_similarity_df.head()

,NER_Entity,Ontology_Individual,NER_Class,Ontology_Class,Similarity_Score,object_property,rel_confidence_Score,relation_status
0,egocentric point-of-view,isStrengthensBy,Perspective,ObjectProperty,0.999666,isStrengthensBy,0.78,existing
1,egocentric point-of-view,Testimony,Perspective,Class,0.999533,providesPerspectiveFor,0.82,existing
2,unsupervised deep neural network,LMUMunich,AIMethodology,NamedIndividual,0.999661,isConductedBy,0.78,existing
3,unsupervised deep neural network,AIAssistedDiagnosis,AIMethodology,Output,0.999583,UsesModel,0.85,existing
4,unsupervised deep neural network,ExtractedFromChatGPT,AIMethodology,NamedIndividual,0.999725,usesModel,0.85,existing


In [ ]:
from rdflib import Graph
import pandas as pd

# Load the existing ontology
ontology_path = "/content/drive/MyDrive/KG_data/hi_ontology_final_1.ttl"
g = Graph()
g.parse(ontology_path, format="turtle")

# Extract individuals and their namespaces
individuals = []

for subj in g.subjects():
    # Convert the subject to a string
    subj_str = str(subj)

    # Check for valid URIs and extract namespace and term
    if "#" in subj_str:
        namespace, term = subj_str.rsplit("#", 1)
    elif "/" in subj_str:
        namespace, term = subj_str.rsplit("/", 1)
    else:
        # Handle cases where there is no recognizable namespace delimiter
        namespace, term = "Unknown", subj_str

    # Add only if it's a valid IRI (not a blank node)
    if namespace and term:
        individuals.append((term, namespace))

# Convert the results to a DataFrame
individual_df = pd.DataFrame(individuals, columns=['Term', 'Namespace'])

# Display the DataFrame
print("Extracted Terms and Namespaces:")
print(individual_df.head())


Extracted Terms and Namespaces:
                    Term                                          Namespace
0  InputsMedicalImagesTo  http://www.semanticweb.org/vbr240/ontologies/2...
1          BiasDetection        http://www.example.org/CriticalComplexityML
2             Prosecutor  http://www.semanticweb.org/vbr240/ontologies/2...
3                   BERT              http://www.example.org/chatGPTImpacts
4              testifies  http://www.semanticweb.org/vbr240/ontologies/2...


In [ ]:
individual_df.head()

,Term,Namespace
0,InputsMedicalImagesTo,http://www.semanticweb.org/vbr240/ontologies/2...
1,BiasDetection,http://www.example.org/CriticalComplexityML
2,Prosecutor,http://www.semanticweb.org/vbr240/ontologies/2...
3,BERT,http://www.example.org/chatGPTImpacts
4,testifies,http://www.semanticweb.org/vbr240/ontologies/2...


In [ ]:
individual_df['Namespace'].value_counts()

,count
Namespace,
http://www.semanticweb.org/vbr240/ontologies/2022/4/untitled-ontology-51,1171
http://www.example.org/CriticalComplexityML,347
http://www.example.org/chatGPTImpacts,178
http://www.example.org/EXOSOUL,173
http://www.example.org/Basiyean_Reasoning,144
http://www.example.org/exemplar,126
Unknown,52
http://www.example.org/Basiyean_Reasoning/,2
http://www.semanticweb.org/vbr240/ontologies/2022/4,1


In [ ]:
# Change the paper_id for term1 and term2 paper-1 -> semantics , paper-2 -> Explainable_AI_implementation, paper-3 -> E-Health_Data_Consent, paper-4 -> Decisionsupport_at_relationalAI, paper-5 ->KG_for_HumanMachine_intelligence.

similarity_df['Term_1_Paper_ID'].replace('2305.05422v1.pdf', 'Visualsemantics', inplace=True)
similarity_df['Term_1_Paper_ID'].replace('A_Conceptual_Model_for_Implementing_Explainable_AI.pdf', 'Explainable_AI_implementation', inplace=True)
similarity_df['Term_1_Paper_ID'].replace('Legal-Ethical_Challenges_and_Technological_Solutio.pdf', 'E-Health_Data_Consent', inplace=True)
similarity_df['Term_1_Paper_ID'].replace('Open_Multiple_Adjunct_Decision_Support_at_the_Time.pdf', 'Decisionsupport_at_relationalAI', inplace=True)
similarity_df['Term_1_Paper_ID'].replace('hhai-2022_paper_67.pdf', 'KG_for_HumanMachine_intelligence', inplace=True)


<ipython-input-12-8edf6031277a>:3: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  similarity_df['Term_1_Paper_ID'].replace('2305.05422v1.pdf', 'Visualsemantics', inplace=True)
<ipython-input-12-8edf6031277a>:4: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'd

In [ ]:
similarity_df.head()

,Term_1,Term_2,Term_1_Paper_ID,Term_2_Paper_ID,Term_1_entity,Term_2_entity,Similarity_Score,object_property,rel_confidence_Score,relation_status
0,egocentric point-of-view,taxonomic structure,Visualsemantics,A_Conceptual_Model_for_Implementing_Explainabl...,Perspective,CategoryStructure,0.999049,isAViewOf,0.85,existing
1,egocentric point-of-view,Collaborative diagnosis,Visualsemantics,hhai-2022_paper_67.pdf,Perspective,MedicalRole,0.998061,isAViewOf,0.78,existing
2,semantic gap,visual-semantic embeddings,Visualsemantics,2305.05422v1.pdf,Problem,ImageClassificationModel,0.999452,aimsToMitigate,0.85,existing
3,Hierarchical Visual Semantics,visual-semantic embeddings,Visualsemantics,2305.05422v1.pdf,Topic,ImageClassificationModel,0.998389,coversTopic,0.80,existing
4,Extreme Value Machines,method engineering,Visualsemantics,A_Conceptual_Model_for_Implementing_Explainabl...,System,EngineeringMethod,0.998048,coversTopic,0.85,existing


In [ ]:
similarity_df['Term_1_Paper_ID'].unique()

array(['Visualsemantics', 'Explainable_AI_implementation',
       'E-Health_Data_Consent', 'Decisionsupport_at_relationalAI',
       'KG_for_HumanMachine_intelligence'], dtype=object)

In [ ]:
similarity_df['Term_2_Paper_ID'].replace('2305.05422v1.pdf', 'Visualsemantics', inplace=True)
similarity_df['Term_2_Paper_ID'].replace('A_Conceptual_Model_for_Implementing_Explainable_AI.pdf', 'Explainable_AI_implementation', inplace=True)
similarity_df['Term_2_Paper_ID'].replace('Legal-Ethical_Challenges_and_Technological_Solutio.pdf', 'E-Health_Data_Consent', inplace=True)
similarity_df['Term_2_Paper_ID'].replace('Open_Multiple_Adjunct_Decision_Support_at_the_Time.pdf', 'Decisionsupport_at_relationalAI', inplace=True)
similarity_df['Term_2_Paper_ID'].replace('hhai-2022_paper_67.pdf', 'KG_for_HumanMachine_intelligence', inplace=True)


<ipython-input-15-d7d04277f1e7>:1: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  similarity_df['Term_2_Paper_ID'].replace('2305.05422v1.pdf', 'Visualsemantics', inplace=True)
<ipython-input-15-d7d04277f1e7>:2: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'd

In [ ]:
similarity_df['Term_2_Paper_ID'].unique()

array(['Explainable_AI_implementation',
       'KG_for_HumanMachine_intelligence', 'Visualsemantics',
       'Decisionsupport_at_relationalAI', 'E-Health_Data_Consent'],
      dtype=object)

In [ ]:
similarity_df['Term_1_Paper_ID'].value_counts()

,count
Term_1_Paper_ID,
E-Health_Data_Consent,21
Decisionsupport_at_relationalAI,9
Visualsemantics,8
Explainable_AI_implementation,7
KG_for_HumanMachine_intelligence,3


In [ ]:
similarity_df['Term_2_Paper_ID'].value_counts()

,count
Term_2_Paper_ID,
KG_for_HumanMachine_intelligence,18
E-Health_Data_Consent,13
Decisionsupport_at_relationalAI,10
Explainable_AI_implementation,5
Visualsemantics,2


In [ ]:
similarity_df[['Term_1','Term_2']].value_counts()

Term_1                                   Term_2                                 
AI usability                             Performance improvement                    1
Artificial Intelligence                  Hybrid Intelligence                        1
Causal factors                           Collaborative diagnosis                    1
                                         Disease-Symptom links                      1
                                         Human-centered AI                          1
Collaborative diagnosis                  Disease-Symptom links                      1
Extreme Value Machines                   Human-centered AI                          1
                                         method engineering                         1
                                         multi-omics data sets                      1
Hierarchical Visual Semantics            visual-semantic embeddings                 1
Human-centered AI                        Collaborative diagnosis                    1
                                         Disease-Symptom links                      1
Knowledge Scientists                     Enterprise Knowledge Graph platform        1
                                         Neurosymbolic computing                    1
Machine learning                         graph machine learning                     1
Open system                              Multiple systems                           1
VcPA (value-centered privacy assistant)  Human-centered AI                          1
                                         autonomous digital interactions            1
XAI tools                                System capabilities                        1
consent mechanisms                       layered dynamic consent platforms          1
data intermediaries                      data cooperatives                          1
data minimization                        User satisfaction                          1
digital interactions                     Digital representation                     1
digital tools                            Collaborative diagnosis                    1
                                         Disease-Symptom links                      1
egocentric point-of-view                 Collaborative diagnosis                    1
                                         taxonomic structure                        1
financial services                       Dutch financial services                   1
healthcare                               ISO standards                              1
                                         taxonomies                                 1
human autonomy                           privacy preferences                        1
method engineering                       multi-omics data sets                      1
                                         taxonomies                                 1
personal data                            data processors                            1
privacy assistants                       VcPA (value-centered privacy assistant)    1
                                         layered dynamic consent platforms          1
                                         value-centered privacy assistants          1
privacy choices                          data cooperatives                          1
                                         data intermediaries                        1
scientific research                      biomedical research                        1
semantic gap                             visual-semantic embeddings                 1
social factors                           Causal factors                             1
taxonomic structure                      Disease-Symptom links                      1
unsupervised deep neural network         Collaborative diagnosis                    1
value-centered privacy assistants        Collaborative diagnosis                    1
                                         Human-ce

In [ ]:
# Combine Term_1 and Term_2 along with their paper IDs and entity classes
combined_df = pd.concat([
    similarity_df[['Term_1', 'Term_1_Paper_ID', 'Term_1_entity']].rename(
        columns={'Term_1': 'Term', 'Term_1_Paper_ID': 'Paper_ID', 'Term_1_entity': 'Entity_Class'}),
    similarity_df[['Term_2', 'Term_2_Paper_ID', 'Term_2_entity']].rename(
        columns={'Term_2': 'Term', 'Term_2_Paper_ID': 'Paper_ID', 'Term_2_entity': 'Entity_Class'})
])

# Group by unique term and aggregate paper IDs and entity classes
unique_terms_df = combined_df.groupby('Term').agg({
    'Paper_ID': lambda x: ', '.join(set(x)),    # Combine unique paper IDs
    'Entity_Class': lambda x: ', '.join(set(x))  # Combine unique entity classes
}).reset_index()

# Display the resulting DataFrame
unique_terms_df.head()

,Term,Paper_ID,Entity_Class
0,AI usability,Decisionsupport_at_relationalAI,UserEndGoal
1,Artificial Intelligence,Decisionsupport_at_relationalAI,AIMethodology
2,Causal factors,Decisionsupport_at_relationalAI,ProblemCharacterstic
3,Collaborative diagnosis,KG_for_HumanMachine_intelligence,MedicalRole
4,Digital representation,Decisionsupport_at_relationalAI,InformationProcessing


In [ ]:
unique_terms_df.describe()

,Term,Paper_ID,Entity_Class
count,52,52,52
unique,52,5,33
top,AI usability,E-Health_Data_Consent,System
freq,1,20,7


In [ ]:
unique_terms_df.to_csv("/content/drive/MyDrive/KG_data/unique_terms_retrieved.csv", index=False)

In [ ]:
unique_terms_df=pd.read_csv("/content/drive/MyDrive/KG_data/Final_data/unique_terms_retrieved.csv")

In [ ]:
unique_terms_df['Entity_Class'].value_counts()

,count
Entity_Class,
System,7
Sector,3
AIMethodology,3
EthicalDecisionModel,3
Dataset,2
LegalParticipant,2
ResearchStudy,2
InteractionTask,2
MLParadigm,2


In [ ]:
from rdflib import Graph, Namespace, RDF, OWL

# Load ontology
ontology_data = Graph()
ontology_data.parse('/content/drive/MyDrive/KG_data/hi_ontology_final_1.ttl', format='turtle')


# Define namespace (Assuming HI is the correct ontology namespace)
HI = Namespace("http://www.semanticweb.org/hi_ontology#")

# Function to extract local names from URIs
def extract_local_name(uri):
    """Extracts the local name after '#' or last '/' from a URI"""
    return uri.split("#")[-1] if "#" in uri else uri.split("/")[-1]

# Extract existing ontology details
existing_classes = [
    extract_local_name(cls) for cls in ontology_data.subjects(RDF.type, OWL.Class)
]

existing_object_property = [
    extract_local_name(object_property) for object_property in ontology_data.subjects(RDF.type, OWL.ObjectProperty)
]

existing_individual = [
    extract_local_name(individual) for individual in ontology_data.subjects(RDF.type, OWL.NamedIndividual)
]

# Display ontology structure
print(f'Existing Ontology is comprised of:\n Classes: {existing_classes}\n Properties: {existing_object_property}\n Individuals: {existing_individual}')
print(f'Existing Ontology count is:\n Total Classes: {len(existing_classes)}\n Total Properties: {len(existing_object_property)}\n Total Individuals: {len(existing_individual)}')


Existing Ontology is comprised of:
 Classes: ['AccidentalDeath', 'AttemptedCrime', 'ComplicityInMurder', 'CrimeMethod', 'CrimeMotive', 'CrimeScene', 'Evidence', 'EvidenceArgument', 'ForensicAnalysis', 'Intent', 'LegalArgument', 'LegalConsideration', 'LegalFact', 'LegalJustification', 'LegalPrinciple', 'LegalScheme', 'nb5467ff3e69e4249b8d839e1d193762ab1', 'Manslaughter', 'Murder', 'NegligentHomicide', 'PhysicalEvidence', 'Testimony', 'VictimSuspectRelationship', 'Weapon', 'WeaponPosession', 'BayesianFragment', 'ComplexProblem', 'GameModelingSystem', 'CulturalConsiderations', 'EthicalDecisionModel', 'EthicalExperiment', 'EthicsResearcher', 'ExplainabilityModel', 'JustificationParameters', 'LegalConsideration', 'PersonalValues', 'ProposedAction', 'ChatGPTEducationalConsequence', 'CommunicationChannel', 'SearchEngine', 'Teacher', 'SkinLesion', 'AIMethodology', 'AIPractitioner', 'AIRefinementProcess', 'Accomplice', 'Alibi', 'AnswerMethod', 'ArtificialAgentEndGoal', 'BayesianLayer', 'Cancer'

In [ ]:
import pandas as pd
from rdflib import Graph, Namespace, RDF, OWL

# Load ontology
ontology_data = Graph()
ontology_data.parse('/content/drive/MyDrive/KG_data/hi_ontology_final_1.ttl', format='turtle')

# Function to extract local names from URIs
def extract_local_name(uri):
    """Extracts the local name after '#' or last '/' from a URI"""
    return uri.split("#")[-1] if "#" in uri else uri.split("/")[-1]

# Lists to store classes and namespaces
classes = []
namespaces = []

# Extract existing ontology classes with their namespaces
for cls in ontology_data.subjects(RDF.type, OWL.Class):
    class_name = extract_local_name(cls)
    namespace = str(cls).replace(class_name, "")
    classes.append(class_name)
    namespaces.append(namespace)

# Create a DataFrame to store the classes and namespaces
existing_classes_df = pd.DataFrame({'Class': classes, 'Namespace': namespaces})

# Display the DataFrame
print("Existing Classes with Namespaces:")
existing_classes_df.head()


Existing Classes with Namespaces:


,Class,Namespace
0,AccidentalDeath,http://www.example.org/Basiyean_Reasoning/
1,AttemptedCrime,http://www.example.org/Basiyean_Reasoning/
2,ComplicityInMurder,http://www.example.org/Basiyean_Reasoning/
3,CrimeMethod,http://www.example.org/Basiyean_Reasoning/
4,CrimeMotive,http://www.example.org/Basiyean_Reasoning/


In [ ]:
existing_classes_df.describe()

,Class,Namespace
count,146,146
unique,145,9
top,LegalConsideration,http://www.semanticweb.org/vbr240/ontologies/2...
freq,2,85


In [ ]:
# Standardize the Entity_Class and Class columns to lowercase and trim spaces
unique_terms_df['Entity_Class_lower'] = unique_terms_df['Entity_Class'].str.lower().str.strip()
existing_classes_df['Class_lower'] = existing_classes_df['Class'].str.lower().str.strip()

# Merge the dataframes on the standardized columns
merged_df = pd.merge(unique_terms_df, existing_classes_df, left_on='Entity_Class_lower', right_on='Class_lower', how='inner')

# Display the merged dataframe
merged_df


,Term,Paper_ID,Entity_Class,Entity_Class_lower,Class,Namespace,Class_lower
0,AI usability,Decisionsupport_at_relationalAI,UserEndGoal,userendgoal,UserEndGoal,http://www.semanticweb.org/vbr240/ontologies/2...,userendgoal
1,Artificial Intelligence,Decisionsupport_at_relationalAI,AIMethodology,aimethodology,AIMethodology,http://www.semanticweb.org/vbr240/ontologies/2...,aimethodology
2,Causal factors,Decisionsupport_at_relationalAI,ProblemCharacterstic,problemcharacterstic,ProblemCharacterstic,http://www.semanticweb.org/vbr240/ontologies/2...,problemcharacterstic
3,Collaborative diagnosis,KG_for_HumanMachine_intelligence,MedicalRole,medicalrole,MedicalRole,http://www.semanticweb.org/vbr240/ontologies/2...,medicalrole
4,Digital representation,Decisionsupport_at_relationalAI,InformationProcessing,informationprocessing,InformationProcessing,http://www.semanticweb.org/vbr240/ontologies/2...,informationprocessing
5,Disease-Symptom links,KG_for_HumanMachine_intelligence,Output,output,Output,http://www.semanticweb.org/vbr240/ontologies/2...,output
6,Enterprise Knowledge Graph platform,KG_for_HumanMachine_intelligence,System,system,System,http://www.semanticweb.org/vbr240/ontologies/2...,system
7,Extreme Value Machines,Visualsemantics,System,system,System,http://www.semanticweb.org/vbr240/ontologies/2...,system
8,Hierarchical Visual Semantics,Visualsemantics,Topic,topic,Topic,http://www.semanticweb.org/vbr240/ontologies/2...,topic
9,Human-centered AI,Decisionsupport_at_relationalAI,AIRefinementProcess,airefinementprocess,AIRefinementProcess,http://www.semanticweb.org/vbr240/ontologies/2...,airefinementprocess


In [ ]:
merged_df.drop(columns=['Entity_Class_lower', 'Class_lower', 'Entity_Class'], inplace=True)

In [ ]:
merged_df.head()

,Term,Paper_ID,Class
0,AI usability,Decisionsupport_at_relationalAI,UserEndGoal
1,Artificial Intelligence,Decisionsupport_at_relationalAI,AIMethodology
2,Causal factors,Decisionsupport_at_relationalAI,ProblemCharacterstic
3,Collaborative diagnosis,KG_for_HumanMachine_intelligence,MedicalRole
4,Digital representation,Decisionsupport_at_relationalAI,InformationProcessing


In [ ]:
merged_df.to_csv("/content/drive/MyDrive/KG_data/merged_df_terms_class.csv", index=False)

## Terms ontology creation:

In [ ]:
from rdflib import Graph, Namespace, RDF, URIRef, Literal
import pandas as pd

# Load the existing ontology file
ontology_graph_ini_1 = Graph()
ontology_graph_ini_1.parse("/content/drive/MyDrive/KG_data/hi_ontology_final_1.ttl", format="turtle")

# Load the merged DataFrame with terms and classes
merged_df_path = "/content/drive/MyDrive/KG_data/Final_data/merged_df_terms_class.csv"
merged_df = pd.read_csv(merged_df_path)

# Display the first few rows of the merged DataFrame to understand its structure
print("First few rows of the merged DataFrame:")
print(merged_df.head())

# Create a new graph to store the updated ontology
updated_graph = Graph()

# Copy existing triples from the original ontology graph to the updated graph
for s, p, o in ontology_graph_ini_1:
    updated_graph.add((s, p, o))

print(f"Copied {len(ontology_graph_ini_1)} triples from the original ontology to the updated graph.")

# Iterate over each row in the merged DataFrame to add new terms
for index, row in merged_df.iterrows():
    term = row['Term'].replace(" ", "_")
    paper_id = row['Paper_ID'].replace(" ", "_")
    class_name = row['Class']
    class_namespace = row['Namespace_y']

    # Create a namespace specifically for this term based on the paper ID
    term_namespace = Namespace(f"http://www.example.org/{paper_id}/")

    # Create URIs for the term and class
    term_uri = term_namespace[term]
    class_uri = URIRef(class_namespace + class_name)

    # Add the new term as an individual under the specified class
    updated_graph.add((term_uri, RDF.type, class_uri))
    updated_graph.add((term_uri, RDF.type, OWL.NamedIndividual))
    updated_graph.add((term_uri, URIRef("http://www.w3.org/2000/01/rdf-schema#label"), Literal(term)))

    print(f"Added individual '{term}' under class '{class_name}' with namespace '{term_namespace}'")

# Print the total number of triples after adding new terms
print(f"\nTotal triples in the updated graph: {len(updated_graph)}")

# Save the updated ontology graph to a new file
updated_ontology_path = "/content/drive/MyDrive/KG_data/updated_hi_ontology_terms_final.ttl"
updated_graph.serialize(destination=updated_ontology_path, format="turtle")
print(f"\nUpdated ontology with terms saved at: {updated_ontology_path}")


First few rows of the merged DataFrame:
                      Term                          Paper_ID  \
0             AI usability   Decisionsupport_at_relationalAI   
1  Artificial Intelligence   Decisionsupport_at_relationalAI   
2           Causal factors   Decisionsupport_at_relationalAI   
3  Collaborative diagnosis  KG_for_HumanMachine_intelligence   
4   Digital representation   Decisionsupport_at_relationalAI   

                   Class                                        Namespace_y  
0            UserEndGoal  http://www.semanticweb.org/vbr240/ontologies/2...  
1          AIMethodology  http://www.semanticweb.org/vbr240/ontologies/2...  
2   ProblemCharacterstic  http://www.semanticweb.org/vbr240/ontologies/2...  
3            MedicalRole  http://www.semanticweb.org/vbr240/ontologies/2...  
4  InformationProcessing  http://www.semanticweb.org/vbr240/ontologies/2...  
Copied 2195 triples from the original ontology to the updated graph.
Added individual 'AI_usability' under 

In [ ]:
# Save the updated ontology graph to a new file
updated_ontology_path = "/content/drive/MyDrive/KG_data/updated_terms_hi_ontology.ttl"
updated_graph.serialize(destination=updated_ontology_path, format="turtle")

<Graph identifier=N1908d056f11c46ceb0bc29db45c190a9 (<class 'rdflib.graph.Graph'>)>

## Relation additon to the Ontology:

In [ ]:
ner_ontology_similarity_df = pd.read_csv("/content/drive/MyDrive/KG_data/Final_data/Relation_dataset_saved_new_new_eo.csv")

In [ ]:
similarity_df = pd.read_csv("/content/drive/MyDrive/KG_data/Final_data/Relation_dataset_saved_new_new.csv")

In [ ]:
#Existing relation in the ontology
# Load ontology
ontology_data = Graph()
ontology_data.parse('/content/drive/MyDrive/KG_data/hi_ontology_final_1.ttl', format='turtle')

# Function to extract local names from URIs
def extract_local_name(uri):
    """Extracts the local name after '#' or last '/' from a URI"""
    return uri.split("#")[-1] if "#" in uri else uri.split("/")[-1]

# Lists to store object properties and namespaces
object_properties = []
namespaces = []

# Extract existing ontology object properties with their namespaces
for obj_prop in ontology_data.subjects(RDF.type, OWL.ObjectProperty):
    obj_prop_name = extract_local_name(obj_prop)
    namespace = str(obj_prop).replace(obj_prop_name, "")
    object_properties.append(obj_prop_name)
    namespaces.append(namespace)

# Create a DataFrame to store the object properties and namespaces
existing_object_properties_df = pd.DataFrame({'ObjectProperty': object_properties, 'Namespace': namespaces})

# Display the DataFrame
print("Existing Object Properties with Namespaces:")
existing_object_properties_df.head()


Existing Object Properties with Namespaces:


,ObjectProperty,Namespace
0,adaptsPreferencesOn,http://www.example.org/EXOSOUL/
1,adaptsToUserPreferencesUsing,http://www.example.org/EXOSOUL/
2,InputsMedicalImagesTo,http://www.semanticweb.org/vbr240/ontologies/2...
3,OfferedAt,http://www.semanticweb.org/vbr240/ontologies/2...
4,OperatesAs,http://www.semanticweb.org/vbr240/ontologies/2...


In [ ]:
similarity_df.head()

,Term_1,Term_2,Term_1_Paper_ID,Term_2_Paper_ID,Term_1_entity,Term_2_entity,Similarity_Score,object_property,rel_confidence_Score,relation_status
0,egocentric point-of-view,taxonomic structure,2305.05422v1.pdf,A_Conceptual_Model_for_Implementing_Explainabl...,Perspective,CategoryStructure,0.999049,isAViewOf,0.85,existing
1,egocentric point-of-view,Collaborative diagnosis,2305.05422v1.pdf,hhai-2022_paper_67.pdf,Perspective,MedicalRole,0.998061,isAViewOf,0.78,existing
2,semantic gap,visual-semantic embeddings,2305.05422v1.pdf,2305.05422v1.pdf,Problem,ImageClassificationModel,0.999452,aimsToMitigate,0.85,existing
3,Hierarchical Visual Semantics,visual-semantic embeddings,2305.05422v1.pdf,2305.05422v1.pdf,Topic,ImageClassificationModel,0.998389,coversTopic,0.80,existing
4,Extreme Value Machines,method engineering,2305.05422v1.pdf,A_Conceptual_Model_for_Implementing_Explainabl...,System,EngineeringMethod,0.998048,coversTopic,0.85,existing


In [ ]:
similarity_df['Term_2_Paper_ID'].replace('2305.05422v1.pdf', 'Visualsemantics', inplace=True)
similarity_df['Term_2_Paper_ID'].replace('A_Conceptual_Model_for_Implementing_Explainable_AI.pdf', 'Explainable_AI_implementation', inplace=True)
similarity_df['Term_2_Paper_ID'].replace('Legal-Ethical_Challenges_and_Technological_Solutio.pdf', 'E-Health_Data_Consent', inplace=True)
similarity_df['Term_2_Paper_ID'].replace('Open_Multiple_Adjunct_Decision_Support_at_the_Time.pdf', 'Decisionsupport_at_relationalAI', inplace=True)
similarity_df['Term_2_Paper_ID'].replace('hhai-2022_paper_67.pdf', 'KG_for_HumanMachine_intelligence', inplace=True)

<ipython-input-37-d7d04277f1e7>:1: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  similarity_df['Term_2_Paper_ID'].replace('2305.05422v1.pdf', 'Visualsemantics', inplace=True)
<ipython-input-37-d7d04277f1e7>:2: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'd

In [ ]:
similarity_df['Term_1_Paper_ID'].replace('2305.05422v1.pdf', 'Visualsemantics', inplace=True)
similarity_df['Term_1_Paper_ID'].replace('A_Conceptual_Model_for_Implementing_Explainable_AI.pdf', 'Explainable_AI_implementation', inplace=True)
similarity_df['Term_1_Paper_ID'].replace('Legal-Ethical_Challenges_and_Technological_Solutio.pdf', 'E-Health_Data_Consent', inplace=True)
similarity_df['Term_1_Paper_ID'].replace('Open_Multiple_Adjunct_Decision_Support_at_the_Time.pdf', 'Decisionsupport_at_relationalAI', inplace=True)
similarity_df['Term_1_Paper_ID'].replace('hhai-2022_paper_67.pdf', 'KG_for_HumanMachine_intelligence', inplace=True)

<ipython-input-38-49972f30f20b>:1: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  similarity_df['Term_1_Paper_ID'].replace('2305.05422v1.pdf', 'Visualsemantics', inplace=True)
<ipython-input-38-49972f30f20b>:2: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'd

In [ ]:
similarity_df.drop(columns=['rel_confidence_Score','Similarity_Score','Term_1_entity','Term_2_entity','rel_confidence_Score'], inplace=True)

In [ ]:
# Standardize the Entity_Class and Class columns to lowercase and trim spaces
similarity_df['object_property_lower'] = similarity_df['object_property'].str.lower().str.strip()
existing_object_properties_df['existing_Object_Property_lower'] = existing_object_properties_df['ObjectProperty'].str.lower().str.strip()

# Merge the dataframes on the standardized columns
merged_df_relation = pd.merge(similarity_df, existing_object_properties_df, left_on='object_property_lower', right_on='existing_Object_Property_lower', how='inner')

# Display the merged dataframe
merged_df_relation.head()


,Term_1,Term_2,Term_1_Paper_ID,Term_2_Paper_ID,object_property,relation_status,object_property_lower,ObjectProperty,Namespace,existing_Object_Property_lower
0,egocentric point-of-view,taxonomic structure,Visualsemantics,Explainable_AI_implementation,isAViewOf,existing,isaviewof,isAViewOf,http://www.semanticweb.org/vbr240/ontologies/2...,isaviewof
1,egocentric point-of-view,Collaborative diagnosis,Visualsemantics,KG_for_HumanMachine_intelligence,isAViewOf,existing,isaviewof,isAViewOf,http://www.semanticweb.org/vbr240/ontologies/2...,isaviewof
2,semantic gap,visual-semantic embeddings,Visualsemantics,Visualsemantics,aimsToMitigate,existing,aimstomitigate,aimsToMitigate,http://www.semanticweb.org/vbr240/ontologies/2...,aimstomitigate
3,Hierarchical Visual Semantics,visual-semantic embeddings,Visualsemantics,Visualsemantics,coversTopic,existing,coverstopic,coversTopic,http://www.semanticweb.org/vbr240/ontologies/2...,coverstopic
4,Extreme Value Machines,method engineering,Visualsemantics,Explainable_AI_implementation,coversTopic,existing,coverstopic,coversTopic,http://www.semanticweb.org/vbr240/ontologies/2...,coverstopic


In [ ]:
merged_df_relation.drop(columns=['object_property_lower', 'existing_Object_Property_lower', 'object_property'], inplace=True)

In [ ]:
merged_df_relation.head()

,Term_1,Term_2,Term_1_Paper_ID,Term_2_Paper_ID,relation_status,ObjectProperty,Namespace
0,egocentric point-of-view,taxonomic structure,Visualsemantics,Explainable_AI_implementation,existing,isAViewOf,http://www.semanticweb.org/vbr240/ontologies/2...
1,egocentric point-of-view,Collaborative diagnosis,Visualsemantics,KG_for_HumanMachine_intelligence,existing,isAViewOf,http://www.semanticweb.org/vbr240/ontologies/2...
2,semantic gap,visual-semantic embeddings,Visualsemantics,Visualsemantics,existing,aimsToMitigate,http://www.semanticweb.org/vbr240/ontologies/2...
3,Hierarchical Visual Semantics,visual-semantic embeddings,Visualsemantics,Visualsemantics,existing,coversTopic,http://www.semanticweb.org/vbr240/ontologies/2...
4,Extreme Value Machines,method engineering,Visualsemantics,Explainable_AI_implementation,existing,coversTopic,http://www.semanticweb.org/vbr240/ontologies/2...


In [ ]:
merged_df_relation.to_csv("/content/drive/MyDrive/KG_data/Final_data/merged_df_relation_final.csv", index=False)

In [ ]:
ner_ontology_similarity_df = pd.read_csv("/content/drive/MyDrive/KG_data/Final_data/Relation_dataset_saved_new_new_eo.csv")

In [ ]:
ner_ontology_similarity_df.head()

,NER_Entity,Ontology_Individual,NER_Class,Ontology_Class,Similarity_Score,object_property,rel_confidence_Score,relation_status
0,egocentric point-of-view,isStrengthensBy,Perspective,ObjectProperty,0.999666,isStrengthensBy,0.78,existing
1,egocentric point-of-view,Testimony,Perspective,Class,0.999533,providesPerspectiveFor,0.82,existing
2,unsupervised deep neural network,LMUMunich,AIMethodology,NamedIndividual,0.999661,isConductedBy,0.78,existing
3,unsupervised deep neural network,AIAssistedDiagnosis,AIMethodology,Output,0.999583,UsesModel,0.85,existing
4,unsupervised deep neural network,ExtractedFromChatGPT,AIMethodology,NamedIndividual,0.999725,usesModel,0.85,existing


In [ ]:
ner_ontology_similarity_df.drop(columns=['rel_confidence_Score','Similarity_Score'], inplace=True)

In [ ]:
# Standardize the Entity_Class and Class columns to lowercase and trim spaces
ner_ontology_similarity_df['object_property_lower'] = ner_ontology_similarity_df['object_property'].str.lower().str.strip()
existing_object_properties_df['existing_Object_Property_lower'] = existing_object_properties_df['ObjectProperty'].str.lower().str.strip()

# Merge the dataframes on the standardized columns
merged_df_relation_on = pd.merge(ner_ontology_similarity_df, existing_object_properties_df, left_on='object_property_lower', right_on='existing_Object_Property_lower', how='inner')

# Display the merged dataframe
merged_df_relation_on.head()


,NER_Entity,Ontology_Individual,NER_Class,Ontology_Class,object_property,relation_status,object_property_lower,ObjectProperty,Namespace,existing_Object_Property_lower
0,egocentric point-of-view,isStrengthensBy,Perspective,ObjectProperty,isStrengthensBy,existing,isstrengthensby,isStrengthensBy,http://www.semanticweb.org/vbr240/ontologies/2...,isstrengthensby
1,egocentric point-of-view,Testimony,Perspective,Class,providesPerspectiveFor,existing,providesperspectivefor,providesPerspectiveFor,http://www.semanticweb.org/vbr240/ontologies/2...,providesperspectivefor
2,unsupervised deep neural network,LMUMunich,AIMethodology,NamedIndividual,isConductedBy,existing,isconductedby,isConductedBy,http://www.semanticweb.org/vbr240/ontologies/2...,isconductedby
3,unsupervised deep neural network,AIAssistedDiagnosis,AIMethodology,Output,UsesModel,existing,usesmodel,usesModel,http://www.semanticweb.org/vbr240/ontologies/2...,usesmodel
4,unsupervised deep neural network,ExtractedFromChatGPT,AIMethodology,NamedIndividual,usesModel,existing,usesmodel,usesModel,http://www.semanticweb.org/vbr240/ontologies/2...,usesmodel


In [ ]:
merged_df_relation_on.drop(columns=['object_property_lower', 'existing_Object_Property_lower', 'object_property',], inplace=True)

In [ ]:
merged_df_relation_on.head()

,NER_Entity,Ontology_Individual,NER_Class,Ontology_Class,relation_status,ObjectProperty,Namespace
0,egocentric point-of-view,isStrengthensBy,Perspective,ObjectProperty,existing,isStrengthensBy,http://www.semanticweb.org/vbr240/ontologies/2...
1,egocentric point-of-view,Testimony,Perspective,Class,existing,providesPerspectiveFor,http://www.semanticweb.org/vbr240/ontologies/2...
2,unsupervised deep neural network,LMUMunich,AIMethodology,NamedIndividual,existing,isConductedBy,http://www.semanticweb.org/vbr240/ontologies/2...
3,unsupervised deep neural network,AIAssistedDiagnosis,AIMethodology,Output,existing,usesModel,http://www.semanticweb.org/vbr240/ontologies/2...
4,unsupervised deep neural network,ExtractedFromChatGPT,AIMethodology,NamedIndividual,existing,usesModel,http://www.semanticweb.org/vbr240/ontologies/2...


In [ ]:
merged_df_relation_on.to_csv("/content/drive/MyDrive/KG_data/Final_data/merged_df_relation_exis_new_final.csv", index=False)

In [ ]:
import pandas as pd
from rdflib import Graph, Namespace, RDF, URIRef, Literal

# Load the merged relation DataFrame
relation_df_path = "/content/drive/MyDrive/KG_data/Final_data/merged_df_relation_final.csv"
relation_df = pd.read_csv(relation_df_path)

# Load the existing updated ontology graph
ontology_graph_updated = Graph()
ontology_graph_updated.parse("/content/drive/MyDrive/KG_data/updated_hi_ontology_terms_final.ttl", format="turtle")

print(f"Loaded ontology graph with {len(ontology_graph_updated)} triples.")

# Track successfully added relations for debugging
added_relations = []

# Iterate over each row in the relation DataFrame to add relationships
for index, row in relation_df.iterrows():
    term_1 = row['Term_1'].replace(" ", "_")
    term_2 = row['Term_2'].replace(" ", "_")
    relation = row['ObjectProperty'].replace(" ", "_")
    paper_id_1 = row['Term_1_Paper_ID'].replace(" ", "_")
    paper_id_2 = row['Term_2_Paper_ID'].replace(" ", "_")
    relation_namespace = row['Namespace'].strip()

    # Create namespaces for each term based on their paper IDs
    namespace_1 = Namespace(f"http://www.example.org/{paper_id_1}/")
    namespace_2 = Namespace(f"http://www.example.org/{paper_id_2}/")

    # Create URIs for the terms and relation
    term_1_uri = namespace_1[term_1]
    term_2_uri = namespace_2[term_2]
    relation_uri = URIRef(relation_namespace + relation)

    # Debugging: Print URIs being created
    print(f"Processing: {term_1_uri} --[{relation_uri}]--> {term_2_uri}")

    # Check if both terms exist in the graph
    if (term_1_uri, RDF.type, None) in ontology_graph_updated and (term_2_uri, RDF.type, None) in ontology_graph_updated:
        # Add the relationship between the existing terms
        ontology_graph_updated.add((term_1_uri, relation_uri, term_2_uri))
        added_relations.append((term_1, relation, term_2))
        print(f"Added relation '{relation}' between '{term_1}' and '{term_2}'")
    else:
        print(f"Skipping relation '{relation}' between '{term_1}' and '{term_2}' as one or both terms do not exist in the graph.")

# Debug: Print all added relations to check
print("\nSuccessfully added relations:")
for rel in added_relations:
    print(f"{rel[0]} --[{rel[1]}]--> {rel[2]}")

# Print graph to see if relations exist
print(f"\nTotal triples in updated graph: {len(ontology_graph_updated)}")
for s, p, o in ontology_graph_updated:
    print(f"Triple: {s} {p} {o}")

# Save the updated graph to a file
output_path = "/content/drive/MyDrive/KG_data/updated_ontology_relations_final.ttl"
ontology_graph_updated.serialize(destination=output_path, format="turtle")
print(f"\nUpdated ontology with relations saved at: {output_path}")


Loaded ontology graph with 2321 triples.
Processing: http://www.example.org/Visualsemantics/egocentric_point-of-view --[http://www.semanticweb.org/vbr240/ontologies/2022/4/untitled-ontology-51#isAViewOf]--> http://www.example.org/Explainable_AI_implementation/taxonomic_structure
Skipping relation 'isAViewOf' between 'egocentric_point-of-view' and 'taxonomic_structure' as one or both terms do not exist in the graph.
Processing: http://www.example.org/Visualsemantics/egocentric_point-of-view --[http://www.semanticweb.org/vbr240/ontologies/2022/4/untitled-ontology-51#isAViewOf]--> http://www.example.org/KG_for_HumanMachine_intelligence/Collaborative_diagnosis
Skipping relation 'isAViewOf' between 'egocentric_point-of-view' and 'Collaborative_diagnosis' as one or both terms do not exist in the graph.
Processing: http://www.example.org/Visualsemantics/semantic_gap --[http://www.semanticweb.org/vbr240/ontologies/2022/4/untitled-ontology-51#aimsToMitigate]--> http://www.example.org/Visualsema

In [ ]:
import pandas as pd
from rdflib import Graph, Namespace, RDF, URIRef, Literal

# Load the merged relation DataFrame
eo_relation_df_path = "/content/drive/MyDrive/KG_data/Final_data/merged_df_relation_exis_new_final.csv"
eo_relation_df = pd.read_csv(eo_relation_df_path)

# Load the existing updated ontology graph
ontology_graph_updated_eo = Graph()
ontology_graph_updated_eo.parse("/content/drive/MyDrive/KG_data/updated_ontology_relations_final.ttl", format="turtle")

print(f"Loaded ontology graph with {len(ontology_graph_updated_eo)} triples.")

# Track successfully added relations for debugging
added_relations = []

# Iterate over each row in the relation DataFrame to add relationships
for index, row in eo_relation_df.iterrows():
    term_1 = row['NER_Entity'].replace(" ", "_")
    term_2 = row['Ontology_Individual'].replace(" ", "_")
    relation = row['ObjectProperty'].replace(" ", "_")
    relation_namespace = row['Namespace'].strip()

    # Find URIs directly from the ontology graph for both terms
    term_1_uri = None
    term_2_uri = None

    # Search for term_1 in the graph
    for s in ontology_graph_updated_eo.subjects(RDF.type, None):
        if str(s).endswith(term_1):
            term_1_uri = s
            break

    # Search for term_2 in the graph
    for s in ontology_graph_updated_eo.subjects(RDF.type, None):
        if str(s).endswith(term_2):
            term_2_uri = s
            break

    # Check if URIs were found
    if term_1_uri is None or term_2_uri is None:
        print(f"Skipping relation '{relation}' between '{term_1}' and '{term_2}' as one or both terms do not exist in the graph.")
        continue

    # Create the relation URI
    relation_uri = URIRef(relation_namespace + relation)

    # Debugging: Print URIs being created
    print(f"Processing: {term_1_uri} --[{relation_uri}]--> {term_2_uri}")

    # Add the relationship between the existing terms
    ontology_graph_updated_eo.add((term_1_uri, relation_uri, term_2_uri))
    added_relations.append((term_1, relation, term_2))
    print(f"Added relation '{relation}' between '{term_1}' and '{term_2}'")

# Debug: Print all added relations to check
print("\nSuccessfully added relations:")
for rel in added_relations:
    print(f"{rel[0]} --[{rel[1]}]--> {rel[2]}")

# Print graph to see if relations exist
print(f"\nTotal triples in updated graph: {len(ontology_graph_updated_eo)}")
for s, p, o in ontology_graph_updated_eo:
    print(f"Triple: {s} {p} {o}")

# Save the updated graph to a file
output_path = "/content/drive/MyDrive/KG_data/updated_ontology_relations_final_eo.ttl"
ontology_graph_updated_eo.serialize(destination=output_path, format="turtle")
print(f"\nUpdated ontology with relations saved at: {output_path}")


Loaded ontology graph with 2358 triples.
Skipping relation 'isStrengthensBy' between 'egocentric_point-of-view' and 'isStrengthensBy' as one or both terms do not exist in the graph.
Skipping relation 'providesPerspectiveFor' between 'egocentric_point-of-view' and 'Testimony' as one or both terms do not exist in the graph.
Processing: http://www.example.org/Visualsemantics/unsupervised_deep_neural_network --[http://www.semanticweb.org/vbr240/ontologies/2022/4/untitled-ontology-51#isConductedBy]--> http://www.example.org/chatGPTImpacts/LMUMunich
Added relation 'isConductedBy' between 'unsupervised_deep_neural_network' and 'LMUMunich'
Processing: http://www.example.org/Visualsemantics/unsupervised_deep_neural_network --[http://www.semanticweb.org/vbr240/ontologies/2022/4/untitled-ontology-51#usesModel]--> http://www.example.org/exemplar/AIAssistedDiagnosis
Added relation 'usesModel' between 'unsupervised_deep_neural_network' and 'AIAssistedDiagnosis'
Processing: http://www.example.org/Vis

In [ ]:
from rdflib import Graph, Namespace, RDF, OWL

# Load ontology
ontology_data = Graph()
ontology_data.parse('/content/drive/MyDrive/KG_data/hi_ontology_final_1.ttl', format='turtle')


# Define namespace (Assuming HI is the correct ontology namespace)
HI = Namespace("http://www.semanticweb.org/hi_ontology#")

# Function to extract local names from URIs
def extract_local_name(uri):
    """Extracts the local name after '#' or last '/' from a URI"""
    return uri.split("#")[-1] if "#" in uri else uri.split("/")[-1]

# Extract existing ontology details
existing_classes = [
    extract_local_name(cls) for cls in ontology_data.subjects(RDF.type, OWL.Class)
]

existing_object_property = [
    extract_local_name(object_property) for object_property in ontology_data.subjects(RDF.type, OWL.ObjectProperty)
]

existing_individual = [
    extract_local_name(individual) for individual in ontology_data.subjects(RDF.type, OWL.NamedIndividual)
]

# Display ontology structure
print(f'Existing Ontology is comprised of:\n Classes: {existing_classes}\n Properties: {existing_object_property}\n Individuals: {existing_individual}')
print(f'Existing Ontology count is:\n Total Classes: {len(existing_classes)}\n Total Properties: {len(existing_object_property)}\n Total Individuals: {len(existing_individual)}')


Existing Ontology is comprised of:
 Classes: ['AccidentalDeath', 'AttemptedCrime', 'ComplicityInMurder', 'CrimeMethod', 'CrimeMotive', 'CrimeScene', 'Evidence', 'EvidenceArgument', 'ForensicAnalysis', 'Intent', 'LegalArgument', 'LegalConsideration', 'LegalFact', 'LegalJustification', 'LegalPrinciple', 'LegalScheme', 'n8179a7c0373547c8988239d485d8e855b1', 'Manslaughter', 'Murder', 'NegligentHomicide', 'PhysicalEvidence', 'Testimony', 'VictimSuspectRelationship', 'Weapon', 'WeaponPosession', 'BayesianFragment', 'ComplexProblem', 'GameModelingSystem', 'CulturalConsiderations', 'EthicalDecisionModel', 'EthicalExperiment', 'EthicsResearcher', 'ExplainabilityModel', 'JustificationParameters', 'LegalConsideration', 'PersonalValues', 'ProposedAction', 'ChatGPTEducationalConsequence', 'CommunicationChannel', 'SearchEngine', 'Teacher', 'SkinLesion', 'AIMethodology', 'AIPractitioner', 'AIRefinementProcess', 'Accomplice', 'Alibi', 'AnswerMethod', 'ArtificialAgentEndGoal', 'BayesianLayer', 'Cancer'

In [ ]:
from rdflib import Graph, Namespace, RDF, OWL

# Load ontology
ontology_data = Graph()
ontology_data.parse('/content/drive/MyDrive/KG_data/updated_hi_ontology_terms_final.ttl', format='turtle')


# Define namespace (Assuming HI is the correct ontology namespace)
HI = Namespace("http://www.semanticweb.org/hi_ontology#")

# Function to extract local names from URIs
def extract_local_name(uri):
    """Extracts the local name after '#' or last '/' from a URI"""
    return uri.split("#")[-1] if "#" in uri else uri.split("/")[-1]

# Extract existing ontology details
existing_classes = [
    extract_local_name(cls) for cls in ontology_data.subjects(RDF.type, OWL.Class)
]

existing_object_property = [
    extract_local_name(object_property) for object_property in ontology_data.subjects(RDF.type, OWL.ObjectProperty)
]

existing_individual = [
    extract_local_name(individual) for individual in ontology_data.subjects(RDF.type, OWL.NamedIndividual)
]

# Display ontology structure
print(f'Existing Ontology is comprised of:\n Classes: {existing_classes}\n Properties: {existing_object_property}\n Individuals: {existing_individual}')
print(f'Existing Ontology count is:\n Total Classes: {len(existing_classes)}\n Total Properties: {len(existing_object_property)}\n Total Individuals: {len(existing_individual)}')
print(f'total triple count: {len(ontology_data)}')


Existing Ontology is comprised of:
 Classes: ['AccidentalDeath', 'AttemptedCrime', 'ComplicityInMurder', 'Manslaughter', 'Murder', 'NegligentHomicide', 'Unreliable_Witness', 'EvidenceArgument', 'LegalConsideration', 'LegalScheme', 'n950714c645e94120ba319a6313a70825b1', 'WeaponPosession', 'CulturalConsiderations', 'LegalConsideration', 'AIRefinementProcess', 'Alibi', 'Cancer', 'EvidenceFragment', 'n950714c645e94120ba319a6313a70825b6', 'EvidenceLayer', 'Innocent', 'OtherOutput', 'QualitativeMethod', 'ResearchStudyEndGoal', 'UserEndGoal', 'ForensicAnalysis', 'LegalPrinciple', 'SearchEngine', 'Accomplice', 'HumanPerformance', 'LegalFragment', 'n950714c645e94120ba319a6313a70825b10', 'LegalLayer', 'LegalRole', 'Patient', 'Victim', 'Interaction', 'Intent', 'PhysicalEvidence', 'EthicalDecisionModel', 'EthicalExperiment', 'ExplainabilityModel', 'Teacher', 'ArtificialAgentEndGoal', 'Discremenator', 'EducationalRole', 'MedicalRole', 'NarrativeLayer', 'ResearchParticipant', 'ResearchRole', 'Resear

In [ ]:
from rdflib import Graph, Namespace, RDF, OWL

# Load ontology
ontology_data = Graph()
ontology_data.parse('/content/drive/MyDrive/KG_data/updated_ontology_relations_final.ttl', format='turtle')


# Define namespace (Assuming HI is the correct ontology namespace)
HI = Namespace("http://www.semanticweb.org/hi_ontology#")

# Function to extract local names from URIs
def extract_local_name(uri):
    """Extracts the local name after '#' or last '/' from a URI"""
    return uri.split("#")[-1] if "#" in uri else uri.split("/")[-1]

# Extract existing ontology details
existing_classes = [
    extract_local_name(cls) for cls in ontology_data.subjects(RDF.type, OWL.Class)
]

existing_object_property = [
    extract_local_name(object_property) for object_property in ontology_data.subjects(RDF.type, OWL.ObjectProperty)
]

existing_individual = [
    extract_local_name(individual) for individual in ontology_data.subjects(RDF.type, OWL.NamedIndividual)
]

# Get total triple count:
total_triples = len(ontology_data)
# Display ontology structure
print(f'Existing Ontology is comprised of:\n Classes: {existing_classes}\n Properties: {existing_object_property}\n Individuals: {existing_individual}')
print(f'Existing Ontology count is:\n Total Classes: {len(existing_classes)}\n Total Properties: {len(existing_object_property)}\n Total Individuals: {len(existing_individual)}')
print(f'Total Triple count is:{total_triples}')


Existing Ontology is comprised of:
 Classes: ['AccidentalDeath', 'AttemptedCrime', 'ComplicityInMurder', 'Manslaughter', 'Murder', 'NegligentHomicide', 'Unreliable_Witness', 'EvidenceArgument', 'LegalConsideration', 'LegalScheme', 'na60280fe6b924d78813aa6980c6d6ed8b1', 'WeaponPosession', 'CulturalConsiderations', 'LegalConsideration', 'AIRefinementProcess', 'Alibi', 'Cancer', 'EvidenceFragment', 'na60280fe6b924d78813aa6980c6d6ed8b6', 'EvidenceLayer', 'Innocent', 'OtherOutput', 'QualitativeMethod', 'ResearchStudyEndGoal', 'UserEndGoal', 'ForensicAnalysis', 'LegalPrinciple', 'SearchEngine', 'Accomplice', 'HumanPerformance', 'LegalFragment', 'na60280fe6b924d78813aa6980c6d6ed8b10', 'LegalLayer', 'LegalRole', 'Patient', 'Victim', 'Interaction', 'Intent', 'PhysicalEvidence', 'EthicalDecisionModel', 'EthicalExperiment', 'ExplainabilityModel', 'Teacher', 'ArtificialAgentEndGoal', 'Discremenator', 'EducationalRole', 'MedicalRole', 'NarrativeLayer', 'ResearchParticipant', 'ResearchRole', 'Resear

In [ ]:
from rdflib import Graph, Namespace, RDF, OWL

# Load ontology
ontology_data = Graph()
ontology_data.parse('/content/drive/MyDrive/KG_data/updated_ontology_relations_final_eo.ttl', format='turtle')


# Define namespace (Assuming HI is the correct ontology namespace)
HI = Namespace("http://www.semanticweb.org/hi_ontology#")

# Function to extract local names from URIs
def extract_local_name(uri):
    """Extracts the local name after '#' or last '/' from a URI"""
    return uri.split("#")[-1] if "#" in uri else uri.split("/")[-1]

# Extract existing ontology details
existing_classes = [
    extract_local_name(cls) for cls in ontology_data.subjects(RDF.type, OWL.Class)
]

existing_object_property = [
    extract_local_name(object_property) for object_property in ontology_data.subjects(RDF.type, OWL.ObjectProperty)
]

existing_individual = [
    extract_local_name(individual) for individual in ontology_data.subjects(RDF.type, OWL.NamedIndividual)
]

# Get total triple count:
total_triples = len(ontology_data)
# Display ontology structure
print(f'Existing Ontology is comprised of:\n Classes: {existing_classes}\n Properties: {existing_object_property}\n Individuals: {existing_individual}')
print(f'Existing Ontology count is:\n Total Classes: {len(existing_classes)}\n Total Properties: {len(existing_object_property)}\n Total Individuals: {len(existing_individual)}')
print(f'Total Triple count is:{total_triples}')


Existing Ontology is comprised of:
 Classes: ['AccidentalDeath', 'AttemptedCrime', 'ComplicityInMurder', 'Manslaughter', 'Murder', 'NegligentHomicide', 'Unreliable_Witness', 'EvidenceArgument', 'LegalConsideration', 'LegalScheme', 'n77c2c35a41e949d38c71aa01c2d37097b1', 'WeaponPosession', 'CulturalConsiderations', 'LegalConsideration', 'AIRefinementProcess', 'Alibi', 'Cancer', 'EvidenceFragment', 'n77c2c35a41e949d38c71aa01c2d37097b6', 'EvidenceLayer', 'Innocent', 'OtherOutput', 'QualitativeMethod', 'ResearchStudyEndGoal', 'UserEndGoal', 'ForensicAnalysis', 'LegalPrinciple', 'SearchEngine', 'HumanPerformance', 'LegalFragment', 'n77c2c35a41e949d38c71aa01c2d37097b10', 'LegalLayer', 'LegalRole', 'Patient', 'Victim', 'Interaction', 'Intent', 'PhysicalEvidence', 'EthicalDecisionModel', 'EthicalExperiment', 'ExplainabilityModel', 'Teacher', 'Accomplice', 'ArtificialAgentEndGoal', 'Discremenator', 'EducationalRole', 'MedicalRole', 'NarrativeLayer', 'ResearchParticipant', 'ResearchRole', 'Resear

In [ ]:
from rdflib import Graph, Namespace, RDF, OWL

# Load ontology
ontology_data = Graph()
ontology_data.parse('/content/drive/MyDrive/KG_data/Data_testing/hi_ontology.ttl', format='turtle')


# Define namespace (Assuming HI is the correct ontology namespace)
HI = Namespace("http://www.semanticweb.org/hi_ontology#")

# Function to extract local names from URIs
def extract_local_name(uri):
    """Extracts the local name after '#' or last '/' from a URI"""
    return uri.split("#")[-1] if "#" in uri else uri.split("/")[-1]

# Extract existing ontology details
existing_classes = [
    extract_local_name(cls) for cls in ontology_data.subjects(RDF.type, OWL.Class)
]

existing_object_property = [
    extract_local_name(object_property) for object_property in ontology_data.subjects(RDF.type, OWL.ObjectProperty)
]

existing_individual = [
    extract_local_name(individual) for individual in ontology_data.subjects(RDF.type, OWL.NamedIndividual)
]

# Get total triple count:
total_triples = len(ontology_data)
# Display ontology structure
print(f'Existing Ontology is comprised of:\n Classes: {existing_classes}\n Properties: {existing_object_property}\n Individuals: {existing_individual}')
print(f'Existing Ontology count is:\n Total Classes: {len(existing_classes)}\n Total Properties: {len(existing_object_property)}\n Total Individuals: {len(existing_individual)}')
print(f'Total Triple count is:{total_triples}')


Existing Ontology is comprised of:
 Classes: ['Context', 'Domain', 'Endgoal', 'EthicalConsideration', 'ArtificialAgent', 'InformationProcessing', 'Human', 'Interaction', 'InteractionTask', 'ProcessingMethod', 'ProcessingTask', 'Actor', 'InteractionMethod', 'Scenario', 'Capability']
 Properties: ['capability', 'context', 'domain', 'endgoal', 'hasEthicalConsideration', 'hasInteraction', 'inScenario', 'informationMethod', 'interactingAgent', 'interactionMethod', 'interactionTask', 'processingInformation', 'processingTask']
 Individuals: ['Adaptiveness', 'Adult', 'BDI', 'Child', 'Collaboration-based', 'Collaborativeness', 'Communication', 'Contribution-based', 'Designer', 'Explainability', 'Generating', 'Hybrid', 'Intelligent_System', 'Learning', 'Multimodal', 'Neuro-symbolic', 'Non-cooperative', 'Question-Answering', 'Reasoning', 'Recommendation', 'Responsibility', 'Robot', 'Single-agent', 'Statistical', 'Symbolic', 'Teacher', 'Transforming', 'Transparency']
Existing Ontology count is:
 T